In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import openai


In [2]:
load_dotenv()

openai.api_key = os.getenv('API_KEY')

In [7]:

column_names = ['versjon', 'avsnitt']  
df = pd.read_json("tolltariffstruktur-kopi.json").loc[:100]

df.head()

,versjon,avsnitt
0,1.5,"{'type': 'avsnitt', 'id': '01', 'beskrivelse':..."
1,1.5,"{'type': 'avsnitt', 'id': '02', 'beskrivelse':..."
2,1.5,"{'type': 'avsnitt', 'id': '03', 'beskrivelse':..."
3,1.5,"{'type': 'avsnitt', 'id': '04', 'beskrivelse':..."
4,1.5,"{'type': 'avsnitt', 'id': '05', 'beskrivelse':..."


In [4]:
df.dropna( inplace=True )
df.shape

(21, 2)

In [8]:
df["VareBeskrivelse"] = df["versjon"].astype(str) + df["avsnitt"].astype(str)
df.head(2)


,versjon,avsnitt,VareBeskrivelse
0,1.5,"{'type': 'avsnitt', 'id': '01', 'beskrivelse':...","1.5{'type': 'avsnitt', 'id': '01', 'beskrivels..."
1,1.5,"{'type': 'avsnitt', 'id': '02', 'beskrivelse':...","1.5{'type': 'avsnitt', 'id': '02', 'beskrivels..."


In [11]:
for i in range(3):
    print(df["versjon"].iloc[i])
    print(df["avsnitt"].iloc[i])
    
    
    

1.5
{'type': 'avsnitt', 'id': '01', 'beskrivelse': 'Levende dyr; animalske produkter', 'kapitler': [{'type': 'kapittel', 'id': '01', 'beskrivelse': 'Levende dyr', 'inndelinger': [{'type': 'posisjon', 'id': '0101', 'beskrivelse': 'Hester, esler, muldyr og mulesler, levende.', 'oppdelinger': [{'type': 'underposisjon', 'beskrivelse': 'hester:', 'hsNummer': '01012', 'oppdelinger': [{'type': 'vare', 'id': '01012100', 'hsNummer': '010121', 'vareslag': 'til avl', 'blv': False, 'henvisninger': ['TNS 001']}, {'type': 'underposisjon', 'beskrivelse': 'andre:', 'hsNummer': '010129', 'oppdelinger': [{'type': 'vare', 'id': '01012902', 'hsNummer': '010129', 'vareslag': 'av vekt under 133 kg', 'blv': False}, {'type': 'vare', 'id': '01012908', 'hsNummer': '010129', 'vareslag': 'ellers', 'blv': False}]}]}, {'type': 'vare', 'id': '01013000', 'hsNummer': '010130', 'vareslag': 'esler', 'blv': False}, {'type': 'vare', 'id': '01019000', 'hsNummer': '010190', 'vareslag': 'andre', 'blv': False}]}, {'type': 'po

In [12]:
import openai

#OpenAI API-nøkkelen
openai.api_key = os.getenv('API_KEY')
#et skrip som vektoriserer dataten og lagrer det. Det blir gjenbrukbart. ! 1 skript: Vektoriser dataen. 2: Lage nytt skriv
#
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = openai.Embedding.create(input=[text], model=model)
    return response['data'][0]['embedding']

In [13]:
sample_embedding = get_embedding("ribbe")

In [14]:
len(sample_embedding)

1536

In [15]:
df["NavnVareBeskrivelse"] = df["VareBeskrivelse"].apply(lambda x: get_embedding(x, model = 'text-embedding-ada-002'))
df.to_json("tolltariffstrukturembedd.csv")

InvalidRequestError: This model's maximum context length is 8192 tokens, however you requested 45114 tokens (45114 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.

In [ ]:
# dele opp filen i 6 og lage batches av det.